Re-implementing the GAIL algorithm on the expert data acquired to run mountain car

In [4]:
import gym
import numpy as np
import os
import gym
import pickle
import argparse
import numpy as np
from collections import deque

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter 
#from utils.utils import *
from utils.zfilter import *
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
torch.autograd.set_detect_anomaly(True)

In [3]:
class Actor(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(num_inputs, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_outputs)
        
        self.fc3.weight.data.mul_(0.1)
        self.fc3.bias.data.mul_(0.0)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        mu = self.fc3(x)
        logstd = torch.zeros_like(mu)
        std = torch.exp(logstd)
        return mu, std


class Critic(nn.Module):
    def __init__(self, num_inputs, hidden_size):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(num_inputs, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        
        self.fc3.weight.data.mul_(0.1)
        self.fc3.bias.data.mul_(0.0)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        v = self.fc3(x)
        return v


class Discriminator(nn.Module):
    def __init__(self, num_inputs, hidden_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(num_inputs, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        
      #  self.fc3.weight.data.mul_(0.1)
     #   self.fc3.bias.data.mul_(0.0)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        prob = torch.sigmoid(self.fc3(x))
        return prob

In [4]:
# I ran a DQN and after it beat the MountainCar env, I told it 

demonstrations = np.load('/Users/noahkasmanoff/Desktop/Projects/lets-do-irl/noah/gail/MCExpert.npy')
                         
demonstrations.shape                  
                         
                         

(37508, 3)

In [ ]:

continuous_expert = []
for i in range(demonstrations.shape[0]):
    episode = []
    for j in range(demonstrations.shape[1]):
        
        demo = demonstrations[i,j,:]

        if np.array_equal(demo[-2:],np.array([2,-1])):
            a = 1.
        else:
            a = -1. 
        continuous_expert.append((demo[0], demo[1],a))
  #  continuous_expert.append(episode)
       # break
        #(state[0], state[1], action)
    


In [ ]:

demonstrations = np.array(continuous_expert)

In [ ]:
demonstrations.shape

In [ ]:
env = gym.make("MountainCarContinuous-v0")
env.reset()
for i in range(demonstrations.shape[0]):
        action = demonstrations[i,-1:]
        s, r, done, _ = env.step(action)
        env.render()
        if done:
            env.close()
            env.reset()
        
       # break
        #(state[0], state[1], action)
    
env.close()

In [ ]:
demonstrations = torch.Tensor(demonstrations)

In [ ]:
index = torch.randperm(demonstrations.shape[0])

In [ ]:
demonstrations[index,:][:16].shape

In [5]:
#code for training 
import torch
import numpy as np
from utils.utils import get_entropy, log_prob_density

def train_discrim(discrim, memory, discrim_optim, demonstrations, discrim_update_num, batch_size, clip_param):
    """
    Training the discriminator. 

    Why? 
    Goal of discrim is to do bce loss on learner and expert.

    Should go through step by step, but the idea is to make learner look bad? That way it improves  

    """
    memory = np.array(memory)  # s a r s' tuple
    states = np.vstack(memory[:, 0]) 
    actions = list(memory[:, 1]) #actions taken by actor/policy

    states = torch.Tensor(states)
    actions = torch.Tensor(actions)
        
    criterion = torch.nn.BCELoss() # classify
 
    for _ in range(discrim_update_num):
        
        learner = discrim(torch.cat([states, actions], dim=1)) #pass (s,a) through discriminator
        demonstrations = torch.Tensor(demonstrations) # pass (s,a) of expert through discriminator
        index = torch.randperm(demonstrations.shape[0])
        demonstrations_batch = demonstrations[index,:][:int(2*batch_size)]
        expert = discrim(demonstrations_batch) #discrimator "guesses" whether or not these 
        # actions came from expert or learner

        discrim_loss = criterion(learner, torch.ones((states.shape[0], 1))) + \
                        criterion(expert, torch.zeros((demonstrations_batch.shape[0], 1)))
                # discrim loss: predict agent is all wrong, get as close to 0, and predict expert is 1, getting as close to 1 as possible. 
        discrim_optim.zero_grad() # gan loss, it tries to always get it right. 
        discrim_loss.backward()
        discrim_optim.step()

        # take these steps, do it however many times specified. 

    expert_acc = ((discrim(demonstrations_batch) < 0.5).float()).mean() #how often it realized the fake examples were fake
    learner_acc = ((discrim(torch.cat([states, actions], dim=1)) > 0.5).float()).mean() #how often if predicted expert correctly. 

    return expert_acc, learner_acc # accuracy, it's the same kind, but because imbalanced better to look at separately. 
 

def train_actor_critic(actor, critic, memory, actor_optim, critic_optim, actor_critic_update_num, batch_size, clip_param):
    """
    Using get gae, this is basically ppo . 

    It's somewhat straightforward, and trained with the irl reward which is 
    from that memory versus what would usually be the real rewa
    """
    memory = np.array(memory) 
    # tuple of a regular old RL problem, but now reward is what the discriminator says. 
    states = np.vstack(memory[:, 0]) 
    actions = list(memory[:, 1]) 
    rewards = list(memory[:, 2])  #IRL Rewards? yes. 
    masks = list(memory[:, 3]) 

    # compute value of what happened, see if what we can get ius better. 
    old_values = critic(torch.Tensor(states))
    #GAE aka estimate of Value + actual return roughtly 
    returns, advants = get_gae(rewards, masks, old_values, gamma, lamda)
    
    # pass states through actor, get corresponding actions
    mu, std = actor(torch.Tensor(states))
    # new mus and stds? 
    old_policy = log_prob_density(torch.Tensor(actions), mu, std) # sum of log probability
    # of old actions

    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)

    for _ in range(actor_critic_update_num):
        np.random.shuffle(arr)

        for i in range(n // batch_size): 
            batch_index = arr[batch_size * i : batch_size * (i + 1)]
            #batch_index = torch.LongTensor(batch_index)
            
            inputs = torch.Tensor(states)[batch_index]
            actions_samples = torch.Tensor(actions)[batch_index]
            returns_samples = returns.unsqueeze(1)[batch_index]
            advants_samples = advants.unsqueeze(1)[batch_index]
            oldvalue_samples = old_values[batch_index].detach()
        
        
            values = critic(inputs) #
            clipped_values = oldvalue_samples + \
                             torch.clamp(values - oldvalue_samples,
                                         -clip_param, 
                                         clip_param)
            critic_loss1 = criterion(clipped_values, returns_samples)
            critic_loss2 = criterion(values, returns_samples)
            critic_loss = torch.max(critic_loss1, critic_loss2).mean()

            loss, ratio, entropy = surrogate_loss(actor, advants_samples, inputs,
                                         old_policy.detach(), actions_samples,
                                         batch_index)
            clipped_ratio = torch.clamp(ratio,
                                        1.0 - clip_param,
                                        1.0 + clip_param)
            clipped_loss = clipped_ratio * advants_samples
            actor_loss = -torch.min(loss, clipped_loss).mean()
            #print(actor_loss,critic_loss,entropy)
           # return actor_loss, critic_loss, entropy
            loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy
           # asdf
            #loss = loss.mean() #TODO
            actor_optim.zero_grad()
            loss.backward()
            actor_optim.step()

           # critic_optim.zero_grad()
           # loss.backward() 
            critic_optim.step()

           # loss.zero_grad()

def get_gae(rewards, masks, values, gamma, lamda):
    """
    How much better a particular action is in a particular state. 
    
    Uses reward of current action + value function of that state-action pair, discount factor gamma, and then lamda to compute. 
    """
    rewards = torch.Tensor(rewards)
    masks = torch.Tensor(masks)
    returns = torch.zeros_like(rewards)
    advants = torch.zeros_like(rewards)
    
    running_returns = 0
    previous_value = 0
    running_advants = 0

    for t in reversed(range(0, len(rewards))):
        running_returns = rewards[t] + (gamma * running_returns * masks[t])
        returns[t] = running_returns

        running_delta = rewards[t] + (gamma * previous_value * masks[t]) - \
                                        values.data[t]
        previous_value = values.data[t]
        
        running_advants = running_delta + (gamma * lamda * \
                                            running_advants * masks[t])
        advants[t] = running_advants

    advants = (advants - advants.mean()) / advants.std()
    return returns, advants

def surrogate_loss(actor, advants, states, old_policy, actions, batch_index):
    """
    The loss for PPO. Re-run through network, recomput policy from states
    and see if this surrogate ratio is better. If it is, use as proximal policy update. It's very close to prior policy, but def better. 
    
    Not sure this actually works though. Should not the new mu and stds be used to draw,
    
        When do we use get_action? Only once in main, I think it should be for all? 
    """
    mu, std = actor(states)
    new_policy = log_prob_density(actions, mu, std)
    old_policy = old_policy[batch_index]

    ratio = torch.exp(new_policy - old_policy)
    surrogate_loss = ratio * advants
    entropy = get_entropy(mu, std)

    return surrogate_loss, ratio, entropy

In [6]:
# Normally args but not here :-)
env_name = 'MountainCarContinuous-v0'
load_model = None
seed = 0
render = False
gamma = 0.99
lamda = .98
hidden_size = 128
train_discrim_flag = True
learning_rate = 3e-4
clip_param = .2
discrim_update_num = 2
actor_critic_update_num = 10
l2_rate = 1e-3 # weight decay
total_sample_size = 256 # total num of state-actions to collect before learning
batch_size = 32
suspend_accu_exp = 1. # do not need to be this high typically, but seems likely it has to be for a simple env like mountain car cont.
suspend_accu_gen = 1.
max_iter_num = 500
seed = 42
logdir = 'logs'

env = gym.make(env_name)
    
env.seed(seed)
torch.manual_seed(seed)

num_inputs = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]
running_state = ZFilter((num_inputs,), clip=5) # huh? 
# oh wow. ZFilter is exactly what I do in capstone project, removing "badtimes"

print('state size:', num_inputs) 
print('action size:', num_actions)

#load agent stuff 
actor = Actor(num_inputs, num_actions, hidden_size)
critic = Critic(num_inputs, hidden_size)
discrim = Discriminator(num_inputs + num_actions, hidden_size)

actor_optim = optim.Adam(actor.parameters(), lr=learning_rate)
critic_optim = optim.Adam(critic.parameters(), lr=learning_rate, 
                          weight_decay=l2_rate) 
discrim_optim = optim.Adam(discrim.parameters(), lr=learning_rate)

state size: 2
action size: 1


In [7]:
# load demonstrations
#expert_demo, _ = pickle.load(open('./expert_demo/expert_demo.p', "rb"))
#demonstrations = np.load('/Users/noahkasmanoff/Desktop/Projects/lets-do-irl/mountaincar/app/expert_demo/expert_demo.npy')
print("demonstrations.shape", demonstrations.shape)

writer = SummaryWriter(logdir)

#if you aren't starting from scratch, load in this 
if load_model is not None:
    saved_ckpt_path = os.path.join(os.getcwd(), 'save_model', str(load_model))
    ckpt = torch.load(saved_ckpt_path)

    # initialize everything
    actor.load_state_dict(ckpt['actor'])
    critic.load_state_dict(ckpt['critic'])
    discrim.load_state_dict(ckpt['discrim'])

    running_state.rs.n = ckpt['z_filter_n']
    running_state.rs.mean = ckpt['z_filter_m']
    running_state.rs.sum_square = ckpt['z_filter_s']

    print("Loaded OK ex. Zfilter N {}".format(running_state.rs.n))

# if no old model no worries, start training. 
episodes = 0
for iter in range(max_iter_num):
    # for i total trajectories 
    actor.eval(), critic.eval()
    memory = deque()

    steps = 0
    scores = []

    while steps < total_sample_size: 
        # sample trajectories  (batch size)
        state = env.reset()
        score = 0

        state = running_state(state) #uh.. again ZFilter related, cleans the state 
        epsteps = 0
        done = False
        while not done: 
            epsteps += 1
            #run through environment
            if render: 
                env.render()

            steps += 1

            mu, std = actor(torch.Tensor(state).unsqueeze(0)) #pass state through actor network
            action = get_action(mu, std)[0] #compute random action
            next_state, reward, done, _ = env.step(action) #take a step
            irl_reward = get_reward(discrim, state, action) #infer what the reward of this action is based on discriminator's get reward 

            if done:
                mask = 0
            else:
                mask = 1 #if done, save this, 

            memory.append([state, action, irl_reward, mask])

            next_state = running_state(next_state) #save cleaned next state
            state = next_state #and set to current state, 

            score += irl_reward #add total reward
           # print("IRL Reward=",irl_reward)
            if done:
                break
            #actual sampling done here 


        
        episodes += 1
        scores.append(score)
        print("steps in ep:", epsteps)
    score_avg = np.mean(scores) #how this model did, 
    print('{}:: {} episode score is {:.2f}'.format(iter, episodes, score_avg))
    writer.add_scalar('log/score', float(score_avg), iter) #logg

    actor.train(), critic.train(), discrim.train() #now train 
    if train_discrim_flag: #if this batch optimizes discrim/reward, 
        # for training the discriminator, classify where state-action pair came from. 
        expert_acc, learner_acc = train_discrim(discrim, memory, discrim_optim, demonstrations, discrim_update_num, batch_size, clip_param) # see comments in train_model. 
        print("Expert: %.2f%% | Learner: %.2f%%" % (expert_acc * 100, learner_acc * 100))
        if expert_acc > suspend_accu_exp and learner_acc > suspend_accu_gen:
            print("Now it will only train the policy, seeing as it is good enough at finding the differences between learner and expert trajectories.")
            train_discrim_flag = False #now train only policy, vanilla RL
            render = True
    #for training actor critic 
    
    # PPO operation, 
    train_actor_critic(actor, critic, memory, actor_optim, critic_optim, actor_critic_update_num, batch_size, clip_param) # no output, see comments in train_model 

    if iter % 100:
        score_avg = int(score_avg)

        model_path = os.path.join(os.getcwd(),'save_model')
        if not os.path.isdir(model_path):
            os.makedirs(model_path)

        ckpt_path = os.path.join(model_path, 'ckpt_'+ str(score_avg)+'.pth.tar')

        save_checkpoint({
            'actor': actor.state_dict(),
            'critic': critic.state_dict(),
            'discrim': discrim.state_dict(),
            'z_filter_n':running_state.rs.n,
            'z_filter_m': running_state.rs.mean,
            'z_filter_s': running_state.rs.sum_square,
           # 'args': args,
            'score': score_avg
        }, filename=ckpt_path)

demonstrations.shape (37508, 3)
steps in ep: 999
0:: 1 episode score is 640.65
Expert: 53.12% | Learner: 64.46%


/Users/noahkasmanoff/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()
/Users/noahkasmanoff/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


steps in ep: 999
1:: 2 episode score is 645.50
Expert: 60.94% | Learner: 63.06%
steps in ep: 999
2:: 3 episode score is 649.56
Expert: 64.06% | Learner: 61.76%
steps in ep: 999
3:: 4 episode score is 661.91
Expert: 87.50% | Learner: 58.96%
steps in ep: 999
4:: 5 episode score is 670.44
Expert: 79.69% | Learner: 55.86%
steps in ep: 505
5:: 6 episode score is 344.94
Expert: 82.81% | Learner: 58.81%
steps in ep: 923
6:: 7 episode score is 628.07
Expert: 90.62% | Learner: 57.20%
steps in ep: 999
7:: 8 episode score is 720.10
Expert: 81.25% | Learner: 51.95%
steps in ep: 999
8:: 9 episode score is 704.85
Expert: 82.81% | Learner: 55.76%
steps in ep: 999
9:: 10 episode score is 690.72
Expert: 87.50% | Learner: 58.16%
steps in ep: 999
10:: 11 episode score is 713.99
Expert: 82.81% | Learner: 55.86%
steps in ep: 999
11:: 12 episode score is 725.76
Expert: 78.12% | Learner: 54.95%
steps in ep: 999
12:: 13 episode score is 746.29
Expert: 92.19% | Learner: 54.05%
steps in ep: 999
13:: 14 episode 

KeyboardInterrupt: 

High reward signal for getting stuck might seem bad, but the discriminator is never going to provide a negative reward value, but instead finds a way to penalize this activity by the fact the the loss is an average over states. By this we know the networks's J (loss) is going to be much lower, and therefore less learning signal via gradient magnitude is sent back. This is a good thing and means more info is given backward for shorter episodes. 

Now that the discriminator has finished training, let's compare how it's reward function looks next to the actual one. 

How should I do this? 

Run trajectories below, print out IRL reward next to reward


In [ ]:


env = gym.make('MountainCarContinuous-v0')
state = env.reset()

In [ ]:
state = env.reset()
done = False
while not done:
        
    mu, std = actor(torch.Tensor(state))
    a = np.tanh(get_action(mu,std))
    state, reward, done, _ = env.step(a)
    print(reward,a)
    env.render()

In [ ]:
env.action_space.sample()

In [ ]:
a